# 导入一分钟线数据

In [ ]:
import os
import json
from datetime import datetime,timedelta
# import mysql.connector
import sqlite3
import shutil

In [ ]:
def datetime_from_str(date,time):
    year_s, mon_s, day_s = date.split('/')
    try:
        return datetime(int(year_s), int(mon_s), int(day_s),int(time[:2]),int(time[2:]))
    except:
        print (year_s, mon_s, day_s,time)

#reading RichMinds config file 
config_file = r'C:\RichMinds\Config.json'
f = open(config_file,'r')
config_json =json.load(f)

#Get data loading path
load_path = config_json['1minute import path']
#print ('股票一分钟线数据导入程序初始化:')
#print ('请确认导入目录:',load_path )
#while True:
#    input_load_path = input('如需更改路径,请输入新的路径')
#    if len(input_load_path) > 5 and os.path.isdir(os.path.join(input_load_path)):
#        load_path = input_load_path
#        break
#    elif input_load_path == '':
#        break
#    print('错误的文件路径')

#Get log path and create log files
log_path = config_json['log path']
current_datetime =datetime.now()
local_now = '{:%Y-%m-%d_%H%M%S}'.format(current_datetime)
log_file_id = '股票1分钟线导入日志_'+ local_now[:19]
log_file_name = log_file_id + '.txt'
log_file_handle = open(os.path.join(log_path,log_file_name),'wt')

#Get processed path for processed file to be moved, path from config plus log_id
processed_path = config_json['processed path'] + '\\'+ log_file_id
os.mkdir(processed_path)

#get file list from data loading path
filenames = [filename for filename in os.listdir(load_path)
               if os.path.isfile(os.path.join(load_path,filename))]

#open database connection, remove MySQL, change to SQLite
#cnx = mysql.connector.connect(user=config_json['DB server']['user'], 
#                              password=config_json['DB server']['password'],
#                              host=config_json['DB server']['host'],
#                              database=config_json['DB server']['database'])

cnx = sqlite3.connect('richmindsdev.db')

dbcursor = cnx.cursor()



print('---开始导入数据---',file = log_file_handle)
#300001 特锐德 1分钟线 不复权
#      日期	    时间	    开盘	    最高	    最低	    收盘	    成交量	    成交额
#2015/12/10,0931,29.61,29.95,29.61,29.83,377100,11201940.00
#2015/12/10,0932,29.84,29.84,29.66,29.70,188000,5592458.00
for filename in filenames:
    print('开始处理文件:',filename,file=log_file_handle)
    filehandler = open(os.path.join(load_path,filename),'rt')
    line_id = 0
    for line in filehandler:
        line_id  = line_id + 1
        line = line.strip()
        if line_id == 1:
            words = line.split(' ')
            stock_id = words[0]
            stock_name = words[1]
#MySQl -> SQLite3
#            add_stock_info = ('''REPLACE INTO stock_basic_info 
#            (Stock_ID, Stock_Name) VALUES (%s, %s)''')
            add_stock_info = '''REPLACE INTO stock_basic_info (Stock_ID, Stock_Name) VALUES (?, ?)'''
            data_stock_info = (stock_id, stock_name)
            # print (add_stock_info,data_stock_info)
            # Insert or replace new stock_info
            dbcursor.execute(add_stock_info,data_stock_info)
        elif line_id ==2: continue
        elif line.startswith('数据来源'): continue
        else:
            words = line.split(',')
            trans_date = words[0]
            trans_time = words[1]
            trans_datetime = datetime_from_str(trans_date,trans_time)
            open_price = words[2]
            high_price = words[3]
            low_price = words[4]
            close_price = words[5]
            trans_volume = words[6]
            trans_turnover = words[7]
#MySQl -> SQLite3 
#            add_stock_trans_detail = ('''INSERT IGNORE INTO stock_1minute_noauth 
#            (Stock_ID,TransDateTime,Open,High,Low,Close,Volume,Turnover)
#            VALUES (%s,%s,%s,%s,%s,%s,%s,%s)''')
            add_stock_trans_detail = '''INSERT OR IGNORE INTO stock_1minute_noauth 
            (Stock_ID,TransDateTime,Open,High,Low,Close,Volume,Turnover)
            VALUES (?,?,?,?,?,?,?,?)'''
            data_stock_trans_detail = (stock_id,trans_datetime,open_price,high_price,low_price,close_price,trans_volume,trans_turnover)
            # Insert or replace new stock_info
            dbcursor.execute( add_stock_trans_detail,data_stock_trans_detail)        
    cnx.commit()
    filehandler.close()
    shutil.move(os.path.join(load_path,filename),os.path.join(processed_path,filename))
        
dbcursor.close()
cnx.close()            
log_file_handle.close()

print ('处理完毕，请检查migration log')


# 基于1分钟线生成N分钟线数据

In [1]:
import os
import json
from datetime import datetime,timedelta
# import mysql.connector
import sqlite3
import shutil

def generateNminutes(Nminutes,StockId,fromdatetime,todatetime):
    cnx = sqlite3.connect('richmindsdev.db')
    dbcursor = cnx.cursor()
    dbupdatecursor = cnx.cursor()
    dbcursor.execute('''SELECT Stock_ID,TransDateTime,Open,High,Low,Close,Volume,Turnover FROM stock_1minute_noauth
                     WHERE Stock_ID = ? and TransDateTime >= ? and TransDateTime <= ? order by TransDateTime''',(StockId,fromdatetime,todatetime) )
        # stock_1minute_data = dbcursor.fetchall()
#DB data sample
#('300002', '2015-12-10 09:31:00', 12.01, 12.01, 11.96, 12, 409800, 4915391)
    if 240 % Nminutes != 0 : #4hours, more than 1 day, it is not allowed
        print ("Nminutes ",Nminutes, "is not allowed!")
        return
    
    open_price = 99999.99
    for stock_1min_row in dbcursor:
        TransDateTime = datetime.strptime(stock_1min_row[1], "%Y-%m-%d %H:%M:%S")
        if int(stock_1min_row[1][11:13]) > 12 and Nmin_TransDateTime.hour < 12: 
            Is_add90min = True
        else:
            Is_add90min = False
            
        if open_price == 99999.99:
            Nmin_TransDateTime = TransDateTime
            open_price = stock_1min_row[2]
            high_price = stock_1min_row[3]
            low_price = stock_1min_row[4]
            close_price = stock_1min_row[5]
            Volume = stock_1min_row[6]
            Turnover  = stock_1min_row[7]  
            continue
            
        if (not Is_add90min and TransDateTime >= (Nmin_TransDateTime + timedelta(minutes = Nminutes)) ) or (Is_add90min and TransDateTime >= (Nmin_TransDateTime + timedelta(minutes = Nminutes+90))):
            #update DB stock_nminutes_noauth
#            print (StockId,Nmin_TransDateTime,open_price,high_price,low_price,close_price,Volume,Turnover)
            add_nmin_trans_detail = '''INSERT OR IGNORE INTO stock_nminute_noauth 
            (Stock_ID,TransDateTime,Nminutes,Open,High,Low,Close,Volume,Turnover)
            VALUES (?,?,?,?,?,?,?,?,?)'''
            data_nmin_trans_detail = (StockId,Nmin_TransDateTime,Nminutes,open_price,high_price,low_price,close_price,
                                      Volume,Turnover)
            # Insert or replace new stock_info
            dbupdatecursor.execute( add_nmin_trans_detail,data_nmin_trans_detail) 
            entry_updated = True
            #initial new nminutes entry
            Nmin_TransDateTime = TransDateTime
            open_price = stock_1min_row[2]
            high_price = stock_1min_row[3]
            low_price = stock_1min_row[4]
            close_price = stock_1min_row[5]
            Volume = stock_1min_row[6]
            Turnover  = stock_1min_row[7]  
        else:
            if high_price < stock_1min_row[3] : high_price = stock_1min_row[3]
            if low_price > stock_1min_row[4]: low_price = stock_1min_row[4]
            close_price = stock_1min_row[5]
            Volume = Volume + stock_1min_row[6]
            Turnover = Turnover + stock_1min_row[7]
            entry_updated = False
    if not entry_updated :
        add_nmin_trans_detail = '''INSERT OR IGNORE INTO stock_nminute_noauth 
            (Stock_ID,TransDateTime,Nminutes,Open,High,Low,Close,Volume,Turnover)
            VALUES (?,?,?,?,?,?,?,?,?)'''
        data_nmin_trans_detail = (StockId,Nmin_TransDateTime,Nminutes,open_price,high_price,low_price,close_price,
                                      Volume,Turnover)
        # Insert or ignore nminutes transaction data into stock_nminutes_noauth
        dbupdatecursor.execute(add_nmin_trans_detail,data_nmin_trans_detail)             
    print (StockId, "processed successfully!")        
    cnx.commit()
    dbcursor.close()
    dbupdatecursor.close()
    cnx.close()

def getstocklist():
    cnx = sqlite3.connect('richmindsdev.db')
    dbcursor = cnx.cursor()
    dbcursor.execute('select Stock_Id from stock_basic_info')
    stocklist = dbcursor.fetchall()
    dbcursor.close()
    cnx.close()
    return stocklist

#stocklist sample:[('300001',), ('300002',), ('300003',), ('300004',), ('300005',), ('300006',), ('300007',)]    
stocklist = getstocklist()
default_fromdatetime = '2015-01-01 09:00:00'
for stock in stocklist:
    generateNminutes(5,stock[0],default_fromdatetime,'2020-01-10 15:00:00')
    

300001 processed successfully!
300002 processed successfully!
300003 processed successfully!
300004 processed successfully!
300005 processed successfully!
300006 processed successfully!
300007 processed successfully!
